In [88]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score, f1_score

In [57]:
df = pd.read_csv('Data_for_UCI_named.csv')

In [58]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [60]:
df.drop('stab', axis= 1, inplace=True)

In [61]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [62]:
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

### Renaming the Traget Column

In [63]:
df['stabf'].replace({'unstable':0,'stable': 1}, inplace= True)

In [64]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,1
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0


In [65]:
# df['stabf'].plot(kind= 'bar')

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stabf   10000 non-null  int64  
dtypes: float64(12), int64(1)
memory usage: 1015.8 KB


## Splitting the dataset

In [67]:
X = df.drop('stabf', axis= 1)
y = df['stabf']

In [68]:
X.head(2)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760


In [69]:
y.head()

0    0
1    1
2    0
3    0
4    0
Name: stabf, dtype: int64

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

## Scaling

In [71]:
from sklearn.preprocessing import StandardScaler

In [72]:
scaler = StandardScaler()

In [74]:
normalised_X_train = scaler.fit_transform(X_train)

In [75]:
y_train = y_train.to_numpy()

In [82]:
y_train = y_train.reshape(-1,1)

In [84]:
# normalised_y_train = scaler.transform(y_train)

In [78]:
normalised_X_test = scaler.transform(X_test)

## Model Building

### Random Forest

In [115]:
from sklearn.ensemble import RandomForestClassifier

randomforestmodel = RandomForestClassifier(random_state=1)

randomforestmodel.fit(normalised_X_train, y_train)

RandomForestClassifier(random_state=1)

In [116]:
# evaluating the model
random_forest_pred = randomforestmodel.predict(normalised_X_test)

In [117]:
random_forest_pred

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [118]:
# Evaluating the accuracy score
rand_acc_score = accuracy_score(y_test, random_forest_pred)

rand_acc_score

0.9295

In [119]:
# Evaluating the f1 score
rand_f1_score = f1_score(y_test, random_forest_pred)

rand_f1_score

0.8980477223427332

## ExtraTreesClassifier

In [87]:
n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {'n_estimators': n_estimators,

'min_samples_leaf': min_samples_leaf,

'min_samples_split': min_samples_split,

'max_features': max_features}

In [96]:
from sklearn.ensemble import ExtraTreesClassifier

extratreeclassifier = ExtraTreesClassifier(random_state=1)

In [127]:
extratreeclassifier.fit(normalised_X_train, y_train)

ExtraTreesClassifier(random_state=1)

In [128]:
# evaluating the model
extratrees_pred = extratreeclassifier.predict(normalised_X_test)

In [129]:
extratrees_pred

array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [130]:
# Evaluating the accuracy score
xtrees_acc_score = accuracy_score(y_test, extratrees_pred)

xtrees_acc_score

0.9285

In [131]:
# Evaluating the f1 score
xtrees_f1_score = f1_score(y_test, extratrees_pred)

xtrees_f1_score

0.8935219657483245

## Xgboost

In [103]:
from xgboost import XGBClassifier 

In [105]:
xgboost = XGBClassifier(random_state=1)

In [112]:
xgboost.fit(normalised_X_train, y_train)

[23:52:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [113]:
model_xgboost = xgboost.predict(normalised_X_test)

In [114]:
xgboost_acc_score = accuracy_score(y_test, model_xgboost)

xgboost_acc_score

0.9455

## Lightgbm

In [122]:
light_gbm = LGBMClassifier(random_state=1)

light_gbm.fit(normalised_X_train, y_train)

LGBMClassifier(random_state=1)

In [123]:
model_lightgbm = light_gbm.predict(normalised_X_test)

In [125]:
lightgbmscore = accuracy_score(y_test, model_lightgbm)

lightgbmscore

0.9395

## Hyperparameter Tuning

In [95]:
from sklearn.model_selection import RandomizedSearchCV

In [132]:
# define search
# search = RandomizedSearchCV(extratreeclassifier, space, n_iter=500, scoring='accuracy', n_jobs=-1, cv=cv, random_state=1)
search = RandomizedSearchCV(extratreeclassifier, hyperparameter_grid)
# execute search
# search2 = RandomizedSearchCV(extratreeclassifier, cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1, random_state = 1)
result = search.fit(X, y)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.933
Best Hyperparameters: {'n_estimators': 1000, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None}


In [134]:
search2 = RandomizedSearchCV(extratreeclassifier,hyperparameter_grid, cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1, random_state = 1)
# execute search

In [135]:
result = search2.fit(X, y)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Score: 0.9275
Best Hyperparameters: {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': None}
